In [1]:
from google.colab import files
uploaded = files.upload()


Saving file.zip to file.zip


In [26]:
import zipfile
import os

zip_path = "file.zip"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("dataset")


os.listdir("dataset")


['file']

In [27]:
import pandas as pd

df = pd.read_csv("dataset/file/train.csv")
df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [28]:
import pandas as pd

df = pd.read_csv("dataset/file/train.csv")

df['label'] = df[['healthy', 'multiple_diseases', 'rust', 'scab']].idxmax(axis=1)
df['image_path'] = "dataset/file/images/" + df['image_id'] + ".jpg"
df = df[['image_id', 'healthy', 'multiple_diseases', 'rust', 'scab', 'image_path', 'label']]
df.head()

,image_id,healthy,multiple_diseases,rust,scab,image_path,label
0,Train_0,0,0,0,1,dataset/file/images/Train_0.jpg,scab
1,Train_1,0,1,0,0,dataset/file/images/Train_1.jpg,multiple_diseases
2,Train_2,1,0,0,0,dataset/file/images/Train_2.jpg,healthy
3,Train_3,0,0,1,0,dataset/file/images/Train_3.jpg,rust
4,Train_4,1,0,0,0,dataset/file/images/Train_4.jpg,healthy


In [29]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

def extract_features(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not load image at {image_path}")
        return None

    image = cv2.resize(image, (128, 128))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    hist_b = cv2.calcHist([image], [0], None, [32], [0, 256]).flatten()
    hist_g = cv2.calcHist([image], [1], None, [32], [0, 256]).flatten()
    hist_r = cv2.calcHist([image], [2], None, [32], [0, 256]).flatten()


    lbp = local_binary_pattern(gray, 8, 1, method="uniform")
    (hist_lbp, _) = np.histogram(lbp.ravel(),
                                 bins=np.arange(0, 10),
                                 range=(0, 9))
    hist_lbp = hist_lbp.astype("float")
    hist_lbp /= (hist_lbp.sum() + 1e-7)


    features = np.hstack([hist_b, hist_g, hist_r, hist_lbp])
    return features

In [30]:
import os

image_directory = "dataset/file/images/"
if os.path.exists(image_directory):
    print(f"Listing files in: {image_directory}")
    for i, filename in enumerate(os.listdir(image_directory)):
        if i < 10:
            print(filename)
        else:
            print("...")
            break
else:
    print(f"Directory not found: {image_directory}")

Listing files in: dataset/file/images/
Train_913.jpg
Train_841.jpg
Train_930.jpg
Train_842.jpg
Train_879.jpg
Train_928.jpg
Train_89.jpg
Train_993.jpg
Train_986.jpg
Train_95.jpg
...


In [ ]:
X = []
y = []

for index, row in df.iterrows():
    features = extract_features(row['image_path'])
    if features is not None: # Only append features if the image was loaded successfully
        X.append(features)
        y.append(row['label'])


X = np.array(X)
y = np.array(y)

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.3333333333333333


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)


In [23]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

model = SVC(kernel='rbf', C=10, gamma=0.01)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))


SVM Accuracy: 0.3333333333333333
                   precision    recall  f1-score   support

          healthy       0.00      0.00      0.00        14
multiple_diseases       0.00      0.00      0.00         1
             rust       0.00      0.00      0.00        13
             scab       0.33      1.00      0.50        14

         accuracy                           0.33        42
        macro avg       0.08      0.25      0.12        42
     weighted avg       0.11      0.33      0.17        42



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [24]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))


Random Forest Accuracy: 0.5238095238095238


In [25]:
!pip install xgboost

from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

print("XGBoost Accuracy:", accuracy_score(y_test, xgb_pred))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:57:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.47619047619047616
